In [14]:
import json
import os
import pandas as pd
import networkx as nx
import numpy as np


In [15]:
class SetEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, set):
            return list(obj)
        return json.JSONEncoder.default(self, obj)


In [16]:
companies = pd.read_csv("data/Heard_Relational_Company_IDs.csv")
locations = pd.read_csv("data/Heard_Relational_Location.csv")
personinfos = pd.read_csv("data/Nodes_researchIT.csv")
edges = pd.read_csv("data/Edges_researchIT.csv")


In [17]:
# personinfos


,ID,Label,Surname,GivenName,Suffix,Gender,Nationality,Employer,Occupation,Location,ValidFrom,ValidTo,Source,Notes
0,2,ID0000001,Heard,Augustine,Sr,Male,American,CO0001,Partner,LC000001,1845,1847,PS000001;PS000003,NaN
1,4,ID0000006,Dixwell,George B,NaN,Male,American,CO0001,Partner,LC000003,1842,1847,PS000001;PS000002;PS000003,NaN
2,6,ID0000002,Heard,John,NaN,Male,American,CO0001,Partner,LC000003,1845,1850,PS000001;PS000003;PS000004;PS000005,NaN
3,7,ID0000008,Roberts,Oliver E,NaN,Male,American,CO0001,Clerk,LC000003,1845,1846,CS137007;PS000001,NaN
4,8,ID0000008,Roberts,Oliver E,NaN,Male,American,CO0003,Clerk,LC000002,1847,1850,CS137007;PS000005,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,268,ID0000102,Farley,Gustavus,Jr,Male,American,CO0020,Agent,LC000015,1876,1876,PS000020,NaN
255,269,ID0000101,Fraser,John Alexander,NaN,Male,British,CO0020,Clerk,LC000015,1876,1876,PS000020,NaN
256,270,ID0000124,Low,O,NaN,Male,NaN,CO0020,Agent,LC000017,1876,1876,PS000020,'F'?
257,271,ID0000126,Livingston,H,NaN,Male,American,CO0020,Clerk,LC000017,1876,1876,PS000020,NaN


In [18]:
companies.CompanyName.fillna('', inplace=True)
companies.DateFounded.fillna('', inplace=True)
companies.DateClosed.fillna('', inplace=True)
companies.NationalAffiliation.fillna('', inplace=True)


In [19]:
person = personinfos[['Label', 'Surname', 'GivenName', 'Suffix',
                      'Gender', 'Nationality', 'Location', 'Employer']].copy()
person = person.drop_duplicates()
person = person.sort_values(by=['Label'])
person.Suffix.fillna('', inplace=True)
person.Nationality.fillna('', inplace=True)


In [20]:
jobs = personinfos[['ID', 'Label', 'Employer', 'Occupation',
                    'Location', 'ValidFrom', 'ValidTo', 'Source', 'Notes']].copy()
jobs.ValidTo.fillna(jobs.ValidFrom, inplace=True)
jobs.Occupation.fillna('', inplace=True)
# jobs.Employer.fillna('CO0000', inplace=True)


In [21]:
job_locations = pd.merge(jobs, locations, how='inner',
                         left_on='Location', right_on='Loc_ID')


In [22]:
job_locations_companies = pd.merge(
    job_locations, companies, how='inner', left_on='Employer', right_on='ID')
job_locations_companies.sort_values(by=['Label'])


,ID_x,Label,Employer,Occupation,Location_x,ValidFrom,ValidTo,Source,Notes,Loc_ID,Location_y,Lat,Lng,ID_y,CompanyName,NationalAffiliation,DateFounded,DateClosed
0,2,ID0000001,CO0001,Partner,LC000001,1845,1847,PS000001;PS000003,NaN,LC000001,Boston,42.360278,-71.057778,CO0001,Augustine Heard & Company,USA,1840.0,1875
4,11,ID0000001,CO0001,Partner,LC000003,1842,1844,PS000002,NaN,LC000003,Guangzhou,23.130000,113.260000,CO0001,Augustine Heard & Company,USA,1840.0,1875
29,46,ID0000002,CO0001,Partner,LC000002,1859,1862,PS000007;PS000008,NaN,LC000002,Hong Kong,22.300000,114.200000,CO0001,Augustine Heard & Company,USA,1840.0,1875
5,12,ID0000002,CO0001,Clerk,LC000003,1842,1844,PS000002,NaN,LC000003,Guangzhou,23.130000,113.260000,CO0001,Augustine Heard & Company,USA,1840.0,1875
223,256,ID0000002,CO0020,Partner,LC000002,1876,1876,PS000020,NaN,LC000002,Hong Kong,22.300000,114.200000,CO0020,Heard & Co,USA,1876.0,1876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,250,ID0000130,CO0001,Clerk,LC000017,1874,1874,PS000018,NaN,LC000017,Kobe,34.690000,135.195556,CO0001,Augustine Heard & Company,USA,1840.0,1875
222,262,ID0000131,CO0020,Agent,LC000003,1876,1876,PS000020,NaN,LC000003,Guangzhou,23.130000,113.260000,CO0020,Heard & Co,USA,1876.0,1876
89,251,ID0000131,CO0001,Clerk,LC000002,1875,1875,CS99217;PS000019,Son of Charles Fearon,LC000002,Hong Kong,22.300000,114.200000,CO0001,Augustine Heard & Company,USA,1840.0,1875
207,254,ID0000132,CO0001,Clerk,LC000017,1875,1875,PS000019,NaN,LC000017,Kobe,34.690000,135.195556,CO0001,Augustine Heard & Company,USA,1840.0,1875


In [23]:
job_locations_companies = pd.merge(job_locations_companies, person[['Label','Surname', 'GivenName', 'Suffix']], how='inner', left_on='Label', right_on='Label')
#job_locations_companies

,ID_x,Label,Employer,Occupation,Location_x,ValidFrom,ValidTo,Source,Notes,Loc_ID,...,Lat,Lng,ID_y,CompanyName,NationalAffiliation,DateFounded,DateClosed,Surname,GivenName,Suffix
0,2,ID0000001,CO0001,Partner,LC000001,1845,1847,PS000001;PS000003,NaN,LC000001,...,42.360278,-71.057778,CO0001,Augustine Heard & Company,USA,1840.0,1875,Heard,Augustine,Sr
1,2,ID0000001,CO0001,Partner,LC000001,1845,1847,PS000001;PS000003,NaN,LC000001,...,42.360278,-71.057778,CO0001,Augustine Heard & Company,USA,1840.0,1875,Heard,Augustine,Sr
2,11,ID0000001,CO0001,Partner,LC000003,1842,1844,PS000002,NaN,LC000003,...,23.130000,113.260000,CO0001,Augustine Heard & Company,USA,1840.0,1875,Heard,Augustine,Sr
3,11,ID0000001,CO0001,Partner,LC000003,1842,1844,PS000002,NaN,LC000003,...,23.130000,113.260000,CO0001,Augustine Heard & Company,USA,1840.0,1875,Heard,Augustine,Sr
4,4,ID0000006,CO0001,Partner,LC000003,1842,1847,PS000001;PS000002;PS000003,NaN,LC000003,...,23.130000,113.260000,CO0001,Augustine Heard & Company,USA,1840.0,1875,Dixwell,George B,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
634,239,ID0000126,CO0001,Clerk,LC000017,1873,1875,PS000017;PS000018;PS000019,Albert Heard's brother-in-law,LC000017,...,34.690000,135.195556,CO0001,Augustine Heard & Company,USA,1840.0,1875,Livingston,H,
635,271,ID0000126,CO0020,Clerk,LC000017,1876,1876,PS000020,NaN,LC000017,...,34.690000,135.195556,CO0020,Heard & Co,USA,1876.0,1876,Livingston,H,
636,271,ID0000126,CO0020,Clerk,LC000017,1876,1876,PS000020,NaN,LC000017,...,34.690000,135.195556,CO0020,Heard & Co,USA,1876.0,1876,Livingston,H,
637,250,ID0000130,CO0001,Clerk,LC000017,1874,1874,PS000018,NaN,LC000017,...,34.690000,135.195556,CO0001,Augustine Heard & Company,USA,1840.0,1875,Gillingham,A,


In [24]:
def strip_dict(d):
    """
    Strip all leading and trailing whitespace in dictionary keys and values.
    """
    return dict((k.strip(), v.strip()) for k, v in d.items())

savepath="json"
os.makedirs(savepath, exist_ok=True)

for year in range(1842, 1892):
    job_locations_companies_year = job_locations_companies[job_locations_companies.ValidFrom.le(
        year) & job_locations_companies.ValidTo.ge(year)].copy()
    companies_subset = companies[companies['ID'].isin(
        set(job_locations_companies_year.ID_y))]
    locations_subset = locations[locations.Loc_ID.isin(
        set(job_locations_companies_year.Loc_ID))]
    ##
    person_subset_builder1 = person[person.Label.isin(
        set(job_locations_companies_year.Label))]
    person_subset = pd.merge(person_subset_builder1, job_locations_companies_year[[
                             'Label', 'Occupation', 'Location_y', 'CompanyName']], how='inner', left_on='Label', right_on='Label')
    # person_subset['ID'] = (person_subset['GivenName'].values + ' ' + person_subset['Surname'].values + ' ' + person_subset['Suffix'].fillna('').astype(str).values);
    # person_subset.drop(columns=['Location', 'Employer'], inplace=True)
    # person_subset.rename(columns={"Location_y": "Location", "CompanyName": "Company"}, inplace=True)
    ##
    edges_subset = edges[(edges.Source.isin(set(job_locations_companies_year.ID_x))) & (
        edges.Target.isin(set(job_locations_companies_year.ID_x)))]

    # Create graph
    G = nx.Graph()
    for n, row in companies_subset.iterrows():
        details = row.to_dict()
        keys = ['ID']
        details = {key: details[key] for key in details if key not in keys}
        details['Company'] = details.pop('CompanyName')
        # G.add_node(row['ID'], node_type="company", details=details)
    for n, row in locations_subset.iterrows():
        details = row.to_dict()
        keys = ['Loc_ID']
        details = {key: details[key] for key in details if key not in keys}
        # G.add_node(row['Location'], node_type="location", details=details)
    for n, row in person_subset.iterrows():
        details = row.to_dict()
        keys = ['Label', 'Employer', 'Location']
        details = {key: details[key] for key in details if key not in keys}
        details['Location'] = details.pop('Location_y')
        details['Company'] = details.pop('CompanyName')
        G.add_node(row['Label'], details=details)
    for n, row in edges_subset.iterrows():
        source_name = job_locations_companies_year[["GivenName", "Surname", "Suffix"]][job_locations_companies_year['ID_x']
                                                       == row.Source].iloc[0]
        sourceName = (source_name['GivenName'] + ' ' + source_name['Surname'] + ' ' + source_name['Suffix']).strip()
        source = job_locations_companies_year['Label'][job_locations_companies_year['ID_x']
                                                       == row.Source].iloc[0]
        target = job_locations_companies_year['Label'][job_locations_companies_year['ID_x']
                                                       == row.Target].iloc[0]
        target_name = job_locations_companies_year[["GivenName", "Surname", "Suffix"]][job_locations_companies_year['ID_x']
                                                       == row.Target].iloc[0]
        targetName = (target_name['GivenName'] + ' ' + target_name['Surname'] + ' ' + target_name['Suffix']).strip()
        
        G.add_edge(source, target, details={
                   'weight', 1}, source_id=sourceName, target_id=targetName)                   
    # Companies
    #for n, row in job_locations_companies_year.iterrows():
    #    emp = row.get('Employer')
    #    if(emp and emp is not np.nan):
    #        pid = row['Label']
    #        m = job_locations_companies_year[(job_locations_companies_year['Label'] == pid) & (
    #            job_locations_companies_year['Employer'] == emp)]
    #        G.add_edge(pid, emp, weight=int(m.size * 5),
    #                   source_type='person', target_type='company')
    # Locations
    #for n, row in job_locations_companies_year.iterrows():
    #    locationy = row.get('Location_y')
    #    if(locationy and locationy is not np.nan):
    #        pid = row['Label']
    #        m = job_locations_companies_year[(job_locations_companies_year['Label'] == pid) & (
    #            job_locations_companies_year['Location_y'] == locationy)]
    #        G.add_edge(pid, locationy, weight=int(m.size * 5),
    #                   source_type='person', target_type='location')
    # Company to Locations
    #for n, row in job_locations_companies_year.iterrows():
    #    locationy = row.get('Location_y')
    #    emp = row.get('Employer')
    #    if(emp and emp is not np.nan and locationy and locationy is not np.nan):
    #        m = job_locations_companies_year[(job_locations_companies_year['Employer'] == emp) & (
    #            job_locations_companies_year['Location_y'] == locationy)]
    #        print(emp)
    #        print(locationy)
    #        G.add_edge(emp, locationy, weight=int(m.size * 5),
    #                   source_type='company', target_type='location')
    pos = {}
    # for node in G.nodes(data=True):
    #   nodename = node[0]
    #   print(nodename)
    #   if node[1]['node_type'] == 'company':
    #     print(node[1])
    #   if node[1]['node_type'] == 'location':
    #     print(node[1])
    #   if node[1]['node_type'] == 'person':
    #     print(node[1])

    NodeLabels = pd.Series((person_subset['GivenName'].values + ' ' + person_subset['Surname'].values +
                           ' ' + person_subset['Suffix'].fillna('').astype(str).values), index=person_subset.Label).to_dict()
    NodeLabels = strip_dict(NodeLabels)
    # Add Companies
    NodeLabels.update(pd.Series(
        companies_subset['CompanyName'].values, index=companies_subset.ID).to_dict())
    # Add Locations
    NodeLabels.update(pd.Series(
        locations_subset['Location'].values, index=locations_subset.Location).to_dict())
    G = nx.relabel_nodes(G, NodeLabels)
    d = nx.json_graph.node_link_data(G)
    converted = json.dumps(d, cls=SetEncoder, indent=4)
    with open(os.path.join(savepath, str(year) + '.json'), 'w') as f:
        f.write(converted)
    G.clear()


In [25]:
distinctCompanies = job_locations_companies['CompanyName'].dropna(
).unique().tolist()
distinctOccupations = job_locations_companies['Occupation'].dropna(
).unique().tolist()
distinctLocations = job_locations_companies['Location_y'].dropna(
).unique().tolist()
distinctNationalities = person['Nationality'].dropna(
).unique().tolist()
distinctNationalAffiliations = job_locations_companies['NationalAffiliation'].dropna(
).unique().tolist()

distinctCompanies.sort()
distinctOccupations.sort()
distinctLocations.sort()
distinctNationalities.sort()
distinctNationalAffiliations.sort()

selections = {
    "companies": distinctCompanies,
    "nationalaffiliations": distinctNationalAffiliations,
    "locations": distinctLocations,
    "occupations": distinctOccupations,
    "nationalities": distinctNationalities
}
#json.dumps(selections)



with open(os.path.join(savepath, "selections.json"), "w") as f:
    json.dump(selections, f, indent=4)


In [26]:
location_json = locations[['Location', 'Lat', 'Lng']].copy().set_index('Location').to_json(orient="index", indent=4)
#print(location_json)
with open(os.path.join(savepath, "locations.json"), "w") as f:
    f.write(location_json)

{
    "Boston":{
        "Lat":42.360278,
        "Lng":-71.057778
    },
    "Hong Kong":{
        "Lat":22.3,
        "Lng":114.2
    },
    "Guangzhou":{
        "Lat":23.13,
        "Lng":113.26
    },
    "Shanghai":{
        "Lat":31.228611,
        "Lng":121.474722
    },
    "Macau":{
        "Lat":22.166667,
        "Lng":113.55
    },
    "London":{
        "Lat":51.507222,
        "Lng":-0.1275
    },
    "Cairo":{
        "Lat":30.044444,
        "Lng":31.235833
    },
    "Monaco":{
        "Lat":43.731111,
        "Lng":7.42
    },
    "Ningbo":{
        "Lat":29.8603,
        "Lng":121.6245
    },
    "Fuzhou":{
        "Lat":26.0743,
        "Lng":119.2964
    },
    "Xiamen":{
        "Lat":24.4796,
        "Lng":118.0889
    },
    "Hankou":{
        "Lat":30.581179,
        "Lng":114.272597
    },
    "Jiujiang":{
        "Lat":29.661,
        "Lng":115.954
    },
    "Tianjin":{
        "Lat":39.1336,
        "Lng":117.2054
    },
    "Yokohama":{
        "Lat":35.4